## Model validation using Hyper parameter, Pipeline and Grid search cross validation

In [14]:
import pandas as pd
import urllib
from urllib import request
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
import numpy as np
import sklearn 
from random import seed
from random import randrange
from math import sqrt
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import r2_score
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from subprocess import check_output
from datetime import time
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn import feature_selection


url = "https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv"
data = pd.read_csv(url)
data['date'] = pd.to_datetime(data['date'])
data['year'], data['month'] , data['time'] , data['hour'] ,data['day'] , data['day_of_week'],data['Numerical_Week'] = data['date'].dt.year, data['date'].dt.month , data['date'].dt.time , data['date'].dt.hour , data['date'].dt.day , data['date'].dt.weekday_name,data['date'].dt.weekday
data['weekStatus'] = data['date'].dt.dayofweek
data['WeekStatus'] = np.where(data['weekStatus'] < 5, 'Weekday', 'Weekend')

d = data.date[0:len(data.date)]
data_final = []
for i in range (len(d)):
    if(i==0):
        a= 61200
        data_final.append(a)
    elif(i>0):
        a=a+600
        data_final.append(a)

data["NSM"] = pd.DataFrame({'NSM':data_final})
data1 = data.set_index('date', 1)
data1 = data1.drop('Appliances', 1)
data1 = data1.drop('day_of_week',1)
data1 = data1.drop('WeekStatus',1)
data1 = data1.drop('time',1)
data1.head(5)

,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,Tdewpoint,rv1,rv2,year,month,hour,day,Numerical_Week,weekStatus,NSM
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,...,5.3,13.275433,13.275433,2016,1,17,11,0,0,61200
2016-01-11 17:10:00,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,...,5.2,18.606195,18.606195,2016,1,17,11,0,0,61800
2016-01-11 17:20:00,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,...,5.1,28.642668,28.642668,2016,1,17,11,0,0,62400
2016-01-11 17:30:00,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,...,5.0,45.410389,45.410389,2016,1,17,11,0,0,63000
2016-01-11 17:40:00,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,...,4.9,10.084097,10.084097,2016,1,17,11,0,0,63600


#### Defining feature matrix(X) and response vector(y)

In [16]:
X = data1
y = data.Appliances
 
# splitting X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,
                                                    random_state=1)

#### Creating Pipeline with feature selection , prepocessing scaler and model estimator

In [4]:
est= RandomForestRegressor()
minmax_scaler = preprocessing.MinMaxScaler()
selector = feature_selection.RFE(est)
pipe_params = [('feat_selection',selector),('std_scaler', minmax_scaler), ('clf', est)]
pipe = Pipeline(pipe_params)

#### Performing Grid search cross validation with hyperparameter tuning

In [6]:
#With the increase in number of parameter execution time is increased 
# param_grid = {'clf__n_estimators' : [100, 50, 200],
#                    'clf__max_features' : ['auto', 'sqrt', 'log2'],
#                   'clf__max_depth': [None, 5, 3, 1] }

# Declare hyperparameters to tune
param_grid = {'clf__n_estimators' : [100, 50, 200] }
clf1 = GridSearchCV(pipe, param_grid=param_grid, cv=10)
clf1.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('feat_selection', RFE(estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight...timators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__n_estimators': [100, 50, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

#### Predicting the target and calculating R sqaure, MAE, RMSE and MAPE

In [20]:
#Doing predictions on training and test data 
predicted_train=clf1.predict(X_train)
predicted_test=clf1.predict(X_test)
#data_test_predicted=reg.predict(data_test1)

#datapredict_test=reg.predict(data_test1)
print("predicted_train : " + str(predicted_train))
print("predicted_test : " + str(predicted_test))
#print("data_test_predicted : " + str(data_test_predicted))
#print("predicted_test : " + str(datapredict_test))


#### MAE Calculation of model
def mae(actual,prediction):
    return mean_absolute_error(actual,prediction)
test_mae=mae(y_test,predicted_test)
train_mae=mae(y_train,predicted_train)
print("test_mae : " + str(test_mae))
print("train_mae : " + str(train_mae))


#### RMSE Calculation of model
def rmse(actual,prediction):
    return np.sqrt(mean_squared_error(actual,prediction))
test_rmse = rmse(y_test,predicted_test)
train_rmse=rmse(y_train,predicted_train)
print("test_rmse : " + str(test_rmse))
print("train_rmse : " + str(train_rmse))

#### R Squared error calculation
test_r2=r2_score(y_test,predicted_test)
train_r2=clf1.score(X_train,y_train)
print("test_r2 : " + str(test_r2))
print("train_r2 : " + str(train_r2))



#### Calculating MAPE
def mean_absolute_percentage_error(y_test,x_predict):
    np.seterr(divide='ignore',invalid='ignore')
    y_test,x_predict=np.array(y_test),np.array(x_predict) 
    return np.mean(np.abs((y_test - x_predict)/y_test))*100   
test_mape = mean_absolute_percentage_error(y_test, predicted_test)
train_mape = mean_absolute_percentage_error(y_train, predicted_train)
print("test_mape : " + str(test_mape))
print("train_mape : " + str(train_mape))


 
# variance score: 1 means perfect prediction
print('Variance score: {}'.format(clf1.score(X_test, y_test)))

predicted_train : [  51.85   33.65   76.   ...,  125.6    61.5    96.05]
predicted_test : [  39.8    60.35  114.4  ...,   95.9   356.05  110.65]
test_mae : 31.1613409897
train_mae : 11.5746054727
test_rmse : 67.8133787658
train_rmse : 25.3815825741
test_r2 : 0.564797904068
train_r2 : 0.938562965494
test_mape : 29.5965124311
train_mape : 11.2258828782
Variance score: 0.56479790406795
